In [1]:
!pip install -r requirements.txt

# Tracking token usage

This notebook goes over how to track your token usage for specific calls. It is currently only implemented for the OpenAI API.

Let's first look at an extremely simple example of tracking token usage for a single LLM call.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = 'Enter your api key here.'

In [3]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [4]:
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [5]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 39
	Prompt Tokens: 4
	Completion Tokens: 35
Successful Requests: 1
Total Cost (USD): $0.0007800000000000001


Anything inside the context manager will get tracked. Here's an example of using it to track multiple calls in sequence.

In [6]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    result2 = llm("Tell me a joke")
    print(cb.total_tokens)

84


If a chain or agent with multiple steps in it is used, it will track all those steps.

In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

os.environ["SERPAPI_API_KEY"] = 'Enter your serp API key here.'

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [9]:
with get_openai_callback() as cb:
    response = agent.run(
        "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new  chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: singer Harry Styles
Thought: I need to find out Harry Styles' age
Action: Search
Action Input: "Harry Styles age"
Observation: 29 years
Thought: I need to calculate 29 raised to the 0.23 power
Action: Calculator
Action Input: 29^0.23
Observation: Answer: 2.169459462491557
Thought: I now know the final answer
Final Answer: Olivia Wilde's boyfriend is Harry Styles and his current age raised to the 0.23 power is 2.169459462491557.

> Finished chain.
Total Tokens: 1422
Prompt Tokens: 1271
Completion Tokens: 151
Total Cost (USD): $0.02844
